# Final Aassignment Applied Economic Analysis

## Introduction

It won´t be long until the billboards will be constructed again, soon to be filled with the faces of local politicians: new elections are close. Wednesday the 16th of March, Dutch citizens will vote for the local municipality elections. Parties and statistical agencies try to predict the elections beforehand using a vast range of methods. Not only are polls and election predictions usefull for measuring the popularity of certain parties, it also helps strategic voters to optimize their votes. Next to that, political parties benefit significantly by knowing where their voters are located: Socialist, for example, will not convince many voters to vote for the Socialist Party when targeting the richest municipalities of the countries. This paper aims to find the relationship between the statisitics of municipalities and the amount of votes on the major Dutch political parties in a structured, automated way. This paper does not aim to suggest a different way of predicting elections as models to predict elections are developped to an extend that cannot be matched with a relatively simple python analysis. 

This paper will use the basic information from municipalities, obtained from the Dutch [Central Bureau of Statistics (CBS)](https://opendata.cbs.nl/statline/portal.html?_la=nl&_catalog=CBS&tableId=70072ned&_theme=233) using the CBS api [cbsodata](https://pypi.org/project/cbsodata/). The election data comes from the Dutch [Election Council (Kiesraad)](https://www.verkiezingsuitslagen.nl/) and has been manually downloaded. I decided not to use [Selenium](https://pypi.org/project/selenium/) and [Beautifulsoup](https://pypi.org/project/beautifulsoup4/) to scrape the data from the website as that would complicate the replicability. The data is obtained per municipality. Although an analysis on the level of neighbourhoods would be more accurate considering it would make the differentiation between e.g. rich and poor, religious and atheistic areas possible, I chose not do this as the availablility of this data is not as good as on municipality level. Obtaining the election data per neighbourhood requires advanced knowledge of XML files and the key figures of the CBS  are either outdated or protected by a paywall.

Because of the large quantity of data used in this project, I chose to work with a database. The parent table will be a table that contains all the municipality codes that have ever existed in the Netherlands between 2000 and 2022. Such a table is necessary as there were many reclassifications of municipalities over the past twenty years. Considering I do not own a server, I decided to use the SQLite dialect for structuring my database. SQLite databases can be stored locally without the need to set up a server. Next to that, SQLite works very well with the pandas pachage, as the function pd.DataFrame.to_sql() and pd.read_sql() can easily interact with the database. 

## Parties

Underneath, the parties incorporated in this research are displayed with their English translation and description. The description has been obtained from [parlement.com](https://www.parlement.com/id/vh8lnhrpfxut/partijen_in_tweede_en_eerste_kamer) and [nsd.no](https://o.nsd.no/european_election_database/country/netherlands/parties.html).  Since the most recent elections in 2021, splitt-offs have caused the number of parties to increase. However, as these parties have participated in clear minority of the elections between 2000 and 2022, they have not been added to this analysis. 

| Party  | English Translation                      | Description                                                                                                                                 |
|--------|------------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------|
| VVD    | People's Party for Freedom and Democracy | Conservative, liberal tradition, strong support of private enterprise and the free market. Right-wing oriented.                             |
| PvdA   | Labour Party                             | Social-democratic, supports shared responsibility, stewardship, justice and solidarity. Left-wing oriented.                                 |
| PVV    | Party for Freedom                        | Conservative, economic liberalism with a conservative  view considering culture and immigration.                                            |
| CDA    | Christian Democratic Appeal              | Combines a centre view towards the economy and defence with conservative leanings. Centre oriented.                                         |
| SP     | Socialist Party                          | Evangelises human dignity, equality and solidarity. Focuses on  employment, social welfare, education and health care. Left-wing oriented.  |
| D66    | Democrats 66                             | Exists of radical democrats and progressive liberals. Centre oriented.                                                                      |
| GL     | Green-Left                               | Promotes democracy, environmental respect, social justice and international solidarity. Left-wing oriented.                                 |
| CU     | Christian Union                          | Conservative considering ethics and social issues. Views on  economic and environmental issues centre-left oriented.                        |
| PvdD   | Party for the Animals                    | Focus on animal welfare and animal rights. Promotes personal freedom and responsibility. Left-wing oriented.                                |
| SGP    | Politically Reformed Party               | Conservative, promotes politics according to biblical values and norms. Right-wing oriented.                                                |
| 50plus |                                          | Focuses on the stakes for people older than 50 years.  Not ideological.                                                                     |
| DENK   |                                          | Focuses on respect for all Dutch inhabitants. Main purpose is to  reach tolerance and a society that is social, learning and righteous.     |
| FvD    | Forum for Democracy                      | Strives for more direct democracy and strengthen the national  sovereignty. Right wing oriented.                                            |

## Obtaining and saving the data

Using SQLite, it is not possible to set foreign keys after the table has been created in the database. Therefore, I need to set these keys manually in an SQLite script. Underneath the function that creates such a SQlite script based on several arguments. *table_name* requires the name you want to give the table in the database. *dataframe* is the table that has to be uploaded to the database. *primary_key* represents the column that should be set as primary, the input is a string. *foreign_key* represents the column that should be set as a foreign key. Note that the input of this parameter is a list, where the first item is the table, the second item the column it references to. *foreign_key* requires the names of the primary key and the foreign key to be the same. Both *primary_key* and *foreign_key* have the value 0 as standard. *Connection* represents the connection to the database, which is locally stored and therefore set to a folder.

In [39]:
import pandas as pd
import sqlite3

def write_dbfile(table_name, dataframe, primary_key = 0,  forkey_ref = 0, connection = 'D:/data/PolProj.db'):
    # Turn editing warning off 
    pd.options.mode.chained_assignment = None
    # Establish sqlite connection
    con = sqlite3.connect(connection)
    # Open cursor
    cur = con.cursor()
    # Obtain table list
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tableframe = pd.DataFrame(cur.fetchall())
    # Table already exists, delete table
    if tableframe.iloc[:,0].str.contains(table_name).any():
        cur.execute(str('DROP TABLE ' + table_name +';'))
    # Opening SQLite statement
    cmd_start = str('CREATE TABLE IF NOT EXISTS ' + table_name + ' ( ')
    # Open dataframe with descriptions of dataframe
    descdat = pd.DataFrame(
        {'colnames': dataframe.columns,
        'coltypes':dataframe.dtypes, 
        'special': float("NaN")}
        )
    # Set index back to numeric 
    descdat = descdat.reset_index(drop=True)
    # If primary key matches with colnames, add to descriptive frame
    try:
        to_primary = descdat.loc[descdat['colnames']== primary_key].index[0]
        descdat.special[to_primary] = 'PRIMARY KEY'
    # If not, give message
    except IndexError: 
        print('No primary key match found')
        pass
    # If foreign key matches with colnames, add to descriptive frame
    if forkey_ref != 0:
        try:
            to_foreign = descdat.loc[descdat['colnames'] == forkey_ref[1]].index[0]
            descdat.special[to_foreign] = 'FOREIGN KEY'
        # if not, give message
        except IndexError: 
            print('No foreign key match found')
            pass
    # Change description column types to strings
    descdat.coltypes = descdat.coltypes.astype(str)
    # Define replacing string function
    clean_string = lambda x,y: descdat.coltypes.str.replace(x, y)
    descdat.coltypes = clean_string('int64', 'INTEGER')
    descdat.coltypes = clean_string('float64', 'REAL')
    descdat.coltypes = clean_string('object', 'TEXT')
    for i,row in descdat.iterrows():
        # Loop over every row and create SQLite command string. 
        if row[2] == 'PRIMARY KEY': 
            cmd_start += str(str(row[0])+ ' '+ str(row[1]) + 'NOT NULL PRIMARY KEY' + ',')
        else: 
            cmd_start += str(str(row[0])+ ' '+ str(row[1]) + ',')
    # If foreign key mentioned, add foreign key indication to SQLite string
    if forkey_ref != 0: 
        cmd_start += str('FOREIGN KEY(' + forkey_ref[1] + ') REFERENCES ' +
                         forkey_ref[0] + '(' + forkey_ref[1]+ '));')
    else: 
        cmd_start = str(cmd_start[:-1]+');')
    # execute string
    cur.execute(cmd_start)    
    # Close cursor and return string for checking 
    cur.close()
    # Write dataframe to newly created table 
    dataframe.to_sql(table_name, con, if_exists = 'append', index = False)
    return()

Now that the function to create relational tables within a dataframe has been defined, the database can be populated. However, the data necessary for this analysis comes from different sources and therefore has to be cleaned. The variable that will link all tables with eachother is the municipality code, used as *municode* from now on. The central table will be a list of all municipality codes in the Netherlands that have existed between 2000 and 2022. The other dataframes will be connected through the central table, as is indicated in the image below: 

![](project_structure.png)

Before data is put in to the database, it needs to be obtained and cleaned. I'll obtain the data from respectively the Central Bureau of Statistics, the Kiesraad starting with the CBS. 

The main table with all the municipality codes that existed between 2000 and 2022 can be obtained through the CBS api and cleaned as follows: 

In [40]:
import cbsodata as cbs 
import pandas as pd
import sqlite3

# Obtain data
muni_all = pd.DataFrame(cbs.get_data('70739ned'))

# Rename columns, set 'municode' as central variable. 
muni_all = muni_all.rename(columns = {'GebiedsOfGemeentecode_3':'municode', 
                                      'EinddatumSorteerveld_7':'enddate'})

# Function for standardising municipality code variable 'municode'
def clean_municode(table):
    remove_string = lambda x: table[~table.RegioS.str.contains(x)]
    # Def strings to remove 
    removables = ['(PV)','(CR)','(LD)','Nederland']
    for item in removables: 
        table = remove_string(item)
    table.dropna(subset = ['municode'], inplace = False)
    clean_string = lambda x: table.municode.str.replace(x, '')
    table.municode = clean_string('GM')
    table.municode = clean_string(' ')
    table.municode = pd.to_numeric(table.municode)
    return(table)

main_data = clean_municode(muni_all)
9
# Table specific cleaning
clean_string = lambda x: main_data.enddate.str.replace(x, '')
main_data.enddate = clean_string(' ')
main_data.enddate = pd.to_numeric(main_data.enddate)
main_data = main_data[(main_data.enddate >= 20000101) | (main_data.enddate.isnull())]
main_data = main_data.drop_duplicates(subset = ['municode'])

## Write data to database
write_dbfile('general_data',main_data,'municode', 0,'D:/data/PolProj.db')

C:\Users\jjhve\AppData\Local\Temp/ipykernel_5300/3144601175.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  remove_string = lambda x: table[~table.RegioS.str.contains(x)]


()

In [ ]:
Now that the parent table has been defined, the other tables can be connected to it. 

In [41]:
# Import general CBS data
cbs_data = pd.DataFrame(cbs.get_data('70072ned'))
# Rename municipality code to general name 
cbs_data = cbs_data.rename(columns = {'KoppelvariabeleRegioCode_306' : 'municode'})

cbs_data = clean_municode(cbs_data)

# Write data to database
write_dbfile('cbs_data', cbs_data, primary_key = 0,forkey_ref = ['general_data', 'municode'], connection = 'D:/data/PolProj.db')

C:\Users\jjhve\AppData\Local\Temp/ipykernel_5300/3144601175.py:14: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  remove_string = lambda x: table[~table.RegioS.str.contains(x)]


No primary key match found


()

In [54]:
import os 
import pandas as pd
# define data map 
def clean_elec():
    # Define directory 
    directory = os.getcwd().replace('\\Functions', '')
    # Import column names + replacements
    colnamers = pd.read_csv(str(directory + '\\colnames.csv'))
    # Create list to filter out small parties (local parties in municipality elections, etc)
    filter_list = list(colnamers.iloc[:,0])
    # Locate data directory 
    directory += str('\\Data')
    for file in os.listdir(directory): 
        # Loop through folders
        directory_temp = str(directory + '\\' + str(file))
        for item in os.listdir(directory_temp):
            # Iterate over csv's in folder
            data = pd.read_csv(str(directory_temp + '\\' + item), sep = ';').rename(columns = {'RegioCode' : 'municode'})
            # Filter out small parties
            data = data[data.columns.intersection(filter_list)]
            # Convert new and old column names to dictionary
            filter_dict = dict(zip(colnamers.colname_old, colnamers.colname_new))
            # Rename columns
            data.rename(columns = filter_dict, inplace = True)
            # Define cleaning function
            remove_signs = lambda x:  data.columns.str.replace(x, '')
            # Define strings to remove
            removables = ['.', '(', ')', '!', '&', '-', ":", '/', '+']  
            # iterate over string list and remove them 
            for character in removables:
                    data.columns = remove_signs(character) 
            # Define string cleaning function
            clean_string = lambda x: data.municode.str.replace(x, '')
            # Filter out everything else except municipalities
            data = data[data['municode'].str.contains('G')]
            data.municode = clean_string('G')
            data.municode = clean_string(' ')
            # Convert municode to numeric
            data.municode = pd.to_numeric(data.municode)
            filename = item.replace('.csv', '')
            # Clean and store data in database
            write_dbfile(table_name = filename, dataframe=data,primary_key = 0,forkey_ref = ['general_data', 'municode'],connection =  'D:/data/PolProj.db', elections = True)
    return(data)
clean_elec()


C:\Users\jjhve\AppData\Local\Temp/ipykernel_5300/1000169532.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  remove_signs = lambda x:  data.columns.str.replace(x, '')


checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found
checkpoint
No primary key match found


,RegioNaam,municode,AmsterdamseCode,OuderRegioNaam,OuderRegioCode,Kiesgerechtigden,Opkomst,OngeldigeStemmen,BlancoStemmen,GeldigeStemmen,...,Oprecht,Jezus_Leeft,DFP,UCF,VSN,PvdE,WZN,PvdR,MN,Groenen
0,Aa en Hunze,1680,10787,Assen,K3,20921,17444,32,25,17387,...,9.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,","
1,Aalsmeer,358,11264,Haarlem,K10,23249,19563,23,35,19505,...,11.0,NaN,12.0,2.0,5.0,NaN,2.0,NaN,NaN,","
2,Aalten,197,11046,Arnhem,K7,21386,17215,18,16,17181,...,4.0,12.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,","
3,Achtkarspelen,59,10199,Leeuwarden,K2,21772,16976,20,27,16929,...,15.0,16.0,NaN,NaN,0.0,NaN,NaN,4.0,NaN,"2,"
4,Alblasserdam,482,11327,Dordrecht,K14,15070,12203,25,16,12162,...,7.0,11.0,NaN,NaN,NaN,NaN,1.0,NaN,1.0,","
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,Zundert,879,11192,Tilburg,K17,17244,13451,23,24,13404,...,4.0,4.0,5.0,0.0,NaN,0.0,NaN,NaN,NaN,","
351,Zutphen,301,10254,Arnhem,K7,37888,30290,51,39,30200,...,12.0,19.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,","
352,Zwartewaterland,1896,10746,Zwolle,K4,16766,14382,38,10,14334,...,3.0,10.0,NaN,2.0,NaN,1.0,1.0,NaN,NaN,","
353,Zwijndrecht,642,10468,Dordrecht,K14,34589,26030,50,47,25933,...,13.0,26.0,NaN,NaN,NaN,NaN,4.0,NaN,3.0,","


In [3]:
import sqlite3
import pandas as pd
from pandas import DataFrame
con = sqlite3.connect('D:/data/PolProj.db')
teststring = 'SELECT * FROM cbs_data'
temp = pd.read_sql(teststring, con)

OperationalError: unable to open database file

In [ ]:
import cbsodata as cbs
coldata = pd.DataFrame(temp.columns, columns = ['colname'])
coldata['partof'] = float('NaN')
cbs_data = pd.DataFrame(cbs.get_meta('70072ned', 'DataProperties'))
coldata.to_csv('coldata.csv')


Notes to self:
Check for unit to see wether it is a percentage. 
Description might indicate what it's based on. Only necessary for explanation. 


In [1]:
cbs_data.iloc[0:300,3:10]

NameError: name 'cbs_data' is not defined

In [4]:
import cbsodata as cbs 
import pandas as pd
import sqlite3

# Use CBS api to obtain the table (link in text above)
muni_all = pd.DataFrame(cbs.get_data('70739ned'))

# Rename municipality code column 
muni_all = muni_all.rename(columns = {'GebiedsOfGemeentecode_3':'municode', 
                                      'EinddatumSorteerveld_7':'enddate'})

# Define function to remove higher levels as provinces, municipality indication 'GM' and empty spaces.
def clean_municode(table, timeclean):
    # Define removal function
    remove_string = lambda x: table[~table.RegioS.str.contains(x)]
    # Define strings to remove 
    removables = ['(PV)','(CR)','(LD)','Nederland']
    # Remove defined strings from column
    for item in removables: 
        table = remove_string(item)
    # Drop empty rows in municode column
    table.dropna(subset = ['municode'], inplace = True)
    # Define string cleaning function
    clean_string = lambda x: table.municode.str.replace(x, '')
    # Clean string with GM and empty space command
    table.municode = clean_string('GM')
    table.municode = clean_string(' ')
    # Transform to numeric 
    table.municode = pd.to_numeric(table.municode)
    if timeclean: 
        # Define cleaning function
        clean_string = lambda x: table.enddate.str.replace(x, '')
        # Remove empty space
        table.enddate = clean_string(' ')
        # Convert to numeric 
        table.enddate = pd.to_numeric(table.enddate)
        # Apply filter to data. 
        table = table[(table.enddate >= 20000101) | (table.enddate.isnull())]
    return(table)

main_data = clean_municode(muni_all, timeclean = True).drop_duplicates(subset = ['municode'])

# Establish database connection
con = sqlite3.connect("D:/data/PolProj.db")

#Write dataframe to database. municode gets primary key 
main_data.to_sql('general_data', con, 
                if_exists = 'replace', index = False, 
                dtype={'municode': 'INTEGER PRIMARY KEY AUTOINCREMENT'})

main_data.head(10)

C:\Users\jjhve\AppData\Local\Temp/ipykernel_14880/1851989226.py:15: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  remove_string = lambda x: table[~table.RegioS.str.contains(x)]


,ID,RegioS,Begindatum_1,Einddatum_2,municode,Provincie_4,ProvincieAfkorting_5,BegindatumSorteerveld_6,enddate
56,56,Aa en Hunze,01 jan 1998,,1680,Drenthe,D.,19980101,NaN
58,58,Aalburg,01 jan 1973,01 jan 2019,738,Noord-Brabant,NB.,19730101,20190101.0
59,59,Aalsmeer,01 jan 1830,,358,Noord-Holland,NH.,18300101,NaN
61,61,Aalten,01 jan 1830,,197,Gelderland,Gld.,18300101,NaN
62,62,Ter Aar,01 jan 1830,01 jan 2007,480,Zuid-Holland,ZH.,18300101,20070101.0
68,68,Abcoude,01 mei 1941,01 jan 2011,305,Utrecht,U.,19410501,20110101.0
73,73,Achtkarspelen,01 jan 1830,,59,Friesland,F.,18300101,NaN
80,80,Akersloot,01 jan 1830,01 jan 2002,360,Noord-Holland,NH.,18300101,20020101.0
81,81,Alblasserdam,01 jan 1830,,482,Zuid-Holland,ZH.,18300101,NaN
82,82,Albrandswaard,01 jan 1985,,613,Zuid-Holland,ZH.,19850101,NaN



This table is obtained from the [CBS](https://opendata.cbs.nl/portal.html?_la=nl&_catalog=CBS&tableId=70739ned&_theme=234) and was created as follows:  